# Feedback tagging - exploratory analysis

Explore the manual feedback tagging data set.

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
from pandas.testing import assert_frame_equal
import numpy as np
import os
import pandas as pd
import re

# Enable multiple cell outputs
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Extract environment variables
DIR_DATA_RAW = os.getenv("DIR_DATA_RAW")

# Define the raw data file path
FILE_RAW = os.path.join(DIR_DATA_RAW, "Coronavirus feedback analysis - Tagging sheet.csv")

# Import the raw data file as a pandas DataFrame
df_raw = pd.read_csv(FILE_RAW)

## Initial preprocessing

Let us do some initial preprocessing of the data. First set all the column headers to lowercase, and replace any punctuation with an underscore (adjacent punctuation will be replaced with a single underscore).

Also drop any rows of data that do not have at least one tag in either the `this_response_relates_to_` or `coronavirus_theme` columns.

In [ ]:
# Make all column names lower case, and replace any punctuation with an underscore
df_preprocess = df_raw.rename(columns=lambda n: re.sub(r"\W+", "_", n.lower()))

# Drop any rows of data where both the `this_response_relates_to_`, and `coronavirus_theme` columns are NA
df_preprocess = df_preprocess[(df_preprocess["this_response_relates_to_"].notnull())|
                              (df_preprocess["coronavirus_theme"].notnull())]

# Set `text_date` as a datetime column - note there are different datetime formats in this column, hence the 
# regular expression parsing to remove anything after YYYY-mm-dd HH:MM:SS
df_preprocess = df_preprocess.assign(text_date=pd.to_datetime(df_preprocess["text_date"].str.replace(
    r"^(?P<one>\d{4}\-\d{2}\-\d{2} \d{2}\:\d{2}\:\d{2}).*$", lambda m: m.group('one'))))

# Show the number of rows dropped
print(f"Initial preprocessing has dropped {len(df_raw) - len(df_preprocess):,} rows ({len(df_raw):,} rows -> "
      f"{len(df_preprocess):,} rows)")
df_preprocess.head()

## Check `text_date` and `numeric_date` columns

Check that the `text_date`, and `numeric_date` columns show the same date.

In [ ]:
pd.to_datetime(df_preprocess["text_date"], utc=True).dt.strftime("%d/%m/%Y").equals(df_preprocess["numeric_date"])

## Examine unique tags

Let us count the number and proportion of unique tags in both the `this_response_relates_to_`, and `coronavirus_theme` columns.

In [ ]:
# Get the counts, and proportions of each unique value in the `this_response_relates_to_` column
df_this_response_relates_to_counts = df_preprocess["this_response_relates_to_"].value_counts(dropna=False) \
    .to_frame("counts")
df_this_response_relates_to_counts = df_this_response_relates_to_counts.assign(
    proportion=df_this_response_relates_to_counts["counts"]/df_preprocess.shape[0]
)

# Get the counts, and proportions of each COVID-19 theme
df_coronavirus_theme_counts = df_preprocess["coronavirus_theme"].value_counts(dropna=False).to_frame("counts")
df_coronavirus_theme_counts = df_coronavirus_theme_counts.assign(
    proportion=df_coronavirus_theme_counts["counts"]/df_preprocess.shape[0]
)

df_this_response_relates_to_counts
df_coronavirus_theme_counts

`df_raw` is sourced from a Google Sheet. In this Google Sheet, there is a tab `Codes` which has a list of possible tags for both the `this_response_relates_to_`, and `coronavirus_theme` columns; let us see what differences `df_preprocess` has between these stated codes (if any).

In [ ]:
# Get the raw `this_response_relates_to` tags that were specificed in the `Codes` sheet of the original raw data
raw_this_response_relates_to = {"govuk-specific", "contact-government", "complaint-government", "service-problem",
                                "address-problem", "finding-problem", "things-problem", "compliment", "ok", "none"}

# Get the raw `coronavirus_themes` tags that were specificed in the `Cods sheet of the original raw data
raw_coronavirus_themes = {"Isolation & quarantine", "Treatment, diagnosis & symptoms", "Lockdown & rules",
                          "Vulnerable", "Other health", "Pregnancy", "Food & deliveries", "Profiteering",
                          "Business loans", "Education and nursery - parents and carers",
                          "Education and nursery - staff", "Health/social care - staff", "Travel & transport - UK",
                          "Travel & transport - abroad", "Livelihood - employee", "Livelihood - self-employed",
                          "Employing others & running a business", "Volunteering", "Housing", 
                          "Advice is confusing, untrustworthy or not up to date", "Privacy concern", 
                          "Data “trustiness” - numbers are unreliable, not up to date or inconsistent", "Other",
                          "Non-COVID"}

# Print the differences between `raw_this_response_relates_to`, and the unique tags we have
print("'df_this_response_relates_to_counts' has the addition unique tags:\n"
      f"\t{set(df_this_response_relates_to_counts.index) - raw_this_response_relates_to}\n")

# Print the differences between `raw_coronavirus_themes`, and the unique tags we have
print("'df_coronavirus_theme_counts' has the addition unique tags:\n"
      f"\t{set(df_coronavirus_theme_counts.index) - raw_coronavirus_themes}")

## Identify duplicated feedback

Let us look at the rows of feedback that have been tagged as `duplicate`.

In [ ]:
# Create a unique key that combines values in both the `this_response_relates_to_` and `coronavirus_theme` columns
df_tag_dup_check = df_preprocess.assign(
    key=df_preprocess["this_response_relates_to_"] + "<-->" + df_preprocess["coronavirus_theme"]
)

# Get all the rows where at least `this_response_relates_to_` or `coronavirus_theme` is a duplicate
df_tag_dup = df_tag_dup_check.query("this_response_relates_to_=='duplicate' or coronavirus_theme=='duplicate'")
df_tag_dup.head()

# Find the rows with duplicates
df_tag_dup_check.query("q3.isin(@df_tag_dup.q3)")

There appear to be three sets of duplicates, but the second set is a set of one value, i.e. there are no duplicates in the data set. I assume this is down to human tagging error.

Let us see if there are more duplicate records in the data set that may **not** have been flagged as `duplicate`.

In [ ]:
# Define some columns to exclude from the duplicate check process - we only want to perform deduplication over the 
# original feedback columns themselves, not any datetime- or taggin-related columns
EXCLUDED_COLUMNS = ["text_date", "this_response_relates_to_", "coronavirus_theme", 
                    "needs_urgent_attention_of_product_teams"]

# Now define the remaining columns
INCLUDED_COLUMNS = [c for c in df_preprocess.columns if c not in EXCLUDED_COLUMNS]

# Sort `df_preprocess` by the earliest `text_date`, then flag any duplicated rows
bool_duplicated_rows = df_preprocess.sort_values(by=["text_date"]) \
    .duplicated(subset=INCLUDED_COLUMNS, keep=False) \
    .sort_index()

# Get the duplicated rows of data
df_dup = df_preprocess[bool_duplicated_rows]
df_dup

There are many more records that appear to be duplicates, but have potentially not been tagged as `duplicate`. Again this is probably due to human error.

If the tags for these duplicates (aside from the `duplicate` tag) are different, then we will need to perform some additional pre-processing to ensure we have the correct data set - let us check this.

In [ ]:
# First drop any duplicates in `df_dup`, aside from in the `text_date` and `needs_urgent_attention_of_product_teams`
# columns. This should give us the unique tags for each of the duplicates
df_dup_diff_tags = df_dup.drop_duplicates(
    subset=[c for c in df_dup.columns if c not in ["text_date"]]
)

# Drop any of `df_dup_diff_tags` that have a `duplicate` tag in either the `this_response_relates_to_` or 
# `coronavirus_theme` columns
df_dup_diff_tags = df_dup_diff_tags.query("this_response_relates_to_ != 'duplicate' or "
                                          "coronavirus_theme != 'duplicate'")

# Now let us extract only those that have different tags - all duplicates with the same tags should now be removed
df_dup_diff_tags = df_dup_diff_tags[df_dup_diff_tags.duplicated(subset=INCLUDED_COLUMNS, keep=False)]

df_dup_diff_tags.shape
df_dup_diff_tags

Let us develop a hierarchy to determine which tag to use if a piece of feedback has multiple tags, as in the seven cases of feedback above.

In [ ]:
# As a reminder, here are the common `this_response_relates_to_`, and `coronavirus_theme` tags
set(df_this_response_relates_to_counts.index) & set(df_coronavirus_theme_counts.index)

Here's how we will define our hierarchy for choosing one amongst duplicated rows:

1. Any tag that is not np.nan, i.e. unfilled;
2. Any tag that is not `duplicate`;
3. Any tag that is not `INTERNAL`;
4. Any tag that is not `none` - only applies for the `this_response_relates_to_` column;
5. Any tag that is not `ok`; and
6. If we still need to break duplicates, then get the earliest datetime (`text_date` column) first.

In [ ]:
# Define a hierarchy of certain `this_response_relates_to_`, and `coronavirus_theme` tags; the keys are the tags, 
# and the values are their 'rank' in descending order, e.g. np.nan is less desirable than `duplicate`. Although 
# 'none' does not appear in `coronavirus_theme`, it is still useful to rank this
ORDER_TAG = {np.nan: -5, "duplicate": -4, "INTERNAL": -3, "none": -2, "ok": -1}

In [ ]:
# Get all boolean series for any rows that have duplicates
bool_duplicate_rows = df_preprocess[INCLUDED_COLUMNS].duplicated(keep=False)

# Get the duplicated rows from `df_preprocess`
df_preprocess_duplicated = df_preprocess[bool_duplicate_rows]

# Rank the datetime for these duplicated rows with earliest datetime last; in descending order, this means the 
# earliest datetime has the largest rank value
s_duplicate_rows_date_ranked = df_preprocess_duplicated["text_date"].rank(method="first", ascending=False)

# Build a `rank` column where `s_duplicate_rows_date_ranked` is used if the `this_response_relates_to_` is not in the
# keys of `ORDER_TAG`. Otherwise, use the value from the mapped dictionary `ORDER_TAG`. This produces a pandas Series 
# with the earliest date having the largest number, unless it maps to `ORDER_TAG`
df_preprocess_duplicated = df_preprocess_duplicated.assign(
    rank_this_response_relates_to=s_duplicate_rows_date_ranked.where(
        ~df_preprocess_duplicated["this_response_relates_to_"].isin(ORDER_TAG.keys()),
        df_preprocess_duplicated["this_response_relates_to_"].replace(ORDER_TAG)).astype(int),
    rank_coronavirus_theme=s_duplicate_rows_date_ranked.where(
        ~df_preprocess_duplicated["coronavirus_theme"].isin(ORDER_TAG.keys()),
        df_preprocess_duplicated["coronavirus_theme"].replace(ORDER_TAG)).astype(int)
)

# Scale the ranks in `df_preprocess_duplicated` so that the minimum value is 1, if the minimum value is less than 0
for col_rank in ["rank_this_response_relates_to", "rank_coronavirus_theme"]:
    if df_preprocess_duplicated[col_rank].min() < 0:
        df_preprocess_duplicated = df_preprocess_duplicated.assign(**{
            col_rank: df_preprocess_duplicated[col_rank] + abs(df_preprocess_duplicated[col_rank].min()) + 1
        })

    
# Create a combined rank using the rank product statistic (geometric mean of the ranks)
df_preprocess_duplicated = df_preprocess_duplicated.assign(
    rank=np.sqrt(df_preprocess_duplicated["rank_this_response_relates_to"].multiply(
        df_preprocess_duplicated["rank_coronavirus_theme"]))
)

# Sort in descending order for the `rank` value first (largest number first), and then alphabetically by all other 
# columns in `INCLUDED_COLUMNS`. Then drop duplicates, keeping only the first entry of any duplicates
df_preprocess_duplicated_ranked = df_preprocess_duplicated \
    .sort_values(by=["rank", *INCLUDED_COLUMNS], ascending=[False, *[True] * len(INCLUDED_COLUMNS)]) \
    .drop_duplicates(subset=INCLUDED_COLUMNS)
df_preprocess_duplicated_ranked.head()

In [ ]:
# Create a finalised, cleaned data set
df_out = pd.concat([df_preprocess[~bool_duplicate_rows], df_preprocess_duplicated_ranked[df_preprocess.columns]]) \
    .sort_index()

# Check if there are any 'duplicate' tags - should be only the one we couldn't find other duplicates for
df_out.query("this_response_relates_to_ == 'duplicate' or coronavirus_theme == 'duplicate'")

# Assert that there are no duplicates in the data
assert_frame_equal(df_out, df_out.drop_duplicates(subset=INCLUDED_COLUMNS)) 

We can probably drop this one remaining `duplicate` tag row, as we do not want to tag duplicate data in our machine learning model - this should be straightforward to do without any modelling.